# Compare power datasets

In [ ]:
# this script compares dataframes producing using the same antenna pattern but with various galactic radio emission models
# first you need to produce these, first run get_power_dataset.py in loop using all the galactic radio emission models
# e.g.: `for m in LFSS GSM08 GSM16 Haslam LFmap SSM GMOSS ULSA; do python get_power_dataset.py -g $m -s Salla_EW; done`


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from pathlib import Path
import os

In [ ]:
# This ensures Plotly output works in multiple places:
# plotly_mimetype: VS Code notebook UI
# notebook: "Jupyter: Export to HTML" command in VS Code
# See https://plotly.com/python/renderers/#multiple-renderers
import plotly.io as pio

pio.renderers.default = "plotly_mimetype+notebook"

In [ ]:
# some global plot settings
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams["font.weight"] = "bold"
plt.rcParams["font.size"] = 16
plt.rcParams["legend.fontsize"] = 14

plt.rcParams["xtick.major.width"] = 2
plt.rcParams["ytick.major.width"] = 2

plt.rcParams["xtick.major.size"] = 5
plt.rcParams["ytick.major.size"] = 5

plt.rcParams["xtick.labelsize"] = 14
plt.rcParams["ytick.labelsize"] = 14

In [ ]:
dir_path = "./simulated_power_datasets/"
df_files = [
    os.path.join(dir_path, i) for i in os.listdir(dir_path) if i.endswith(".csv")
]

In [ ]:
df_list = []
df_names = []
for f in df_files:
    df = pd.read_csv(f, index_col=0)
    df.columns = df.columns.astype(float)
    df_list.append(df)
    df_names.append(Path(f).stem)

concatenated_df = pd.concat(df_list, keys=df_names)


In [ ]:
# HINTS
# print df main keys
# print(concatenated_df.index.levels[0])
# get the same cell from each dfs
# one_cell_values = np.asarray([concatenated_df.xs(key).iloc[0,0] for key in concatenated_df.index.levels[0]])


In [ ]:
# mean across the cells of all dfs
mean_df = concatenated_df.groupby(level=1).mean()
# std
std_df = concatenated_df.groupby(level=1).std()


In [ ]:
fig = px.imshow(mean_df.T, width=600, aspect="cube", color_continuous_scale="jet", title='<b>Mean</b>')
fig.update_layout(
    xaxis=dict(title="<b>LST</b>", tickprefix="<b>", ticksuffix="</b>", dtick=2),
    yaxis=dict(
        title="<b>frequency [MHz]</b>",
        tickprefix="<b>",
        ticksuffix="</b>",
        range=(30, 80),
        tick0=0,
        dtick=10,
        autorange=False,
    ),
)
fig.update_layout(
    coloraxis=dict(
        colorbar=dict(
            title=dict(
                text="<b><power> [pW]</b>",
                side="right",
            ),
            tickprefix="<b>",
            ticksuffix="</b>",
        ),
        cmin=0,
        cmax=20,
    ),
    font=dict(
        # family=font,
        size=16,
        color="black",
    ),
)

In [ ]:
fig = px.imshow(
    (std_df / mean_df.values).T, width=600, aspect="cube", color_continuous_scale="jet", title='<b>Relative standard deviation</b>'
)
fig.update_layout(
    xaxis=dict(title="<b>LST</b>", tickprefix="<b>", ticksuffix="</b>", dtick=2),
    yaxis=dict(
        title="<b>frequency [MHz]</b>",
        tickprefix="<b>",
        ticksuffix="</b>",
        range=(30, 80),
        tick0=0,
        dtick=10,
        autorange=False,
    ),
    coloraxis=dict(
        colorbar=dict(
            title=dict(
                text="<b>\u03C3 [%]</b>",
                side="right",
            ),
            tickprefix="<b>",
            ticksuffix="</b>",
        ),
    ),
    font=dict(
        # family=font,
        size=16,
        color="black",
    ),
)

In [ ]:
fig, ax = plt.subplots()

ax.hist((std_df / mean_df.values).values.flatten())
ax.set_xlabel(r"$\sigma$ [%]")
ax.set_ylabel("entries")

In [ ]:
# total mean
np.mean((std_df / mean_df.values).values.flatten())